<a href="https://colab.research.google.com/github/kdkd9/sentiment_analysis/blob/main/Sentiment_analysis_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd

In [4]:
data=pd.read_csv("/content/IMDB Dataset.csv")

In [5]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [6]:
data.columns

Index(['review', 'sentiment'], dtype='object')

In [7]:
data.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [9]:
data.isnull().sum()

review       0
sentiment    0
dtype: int64

In [10]:
data.isnull().any()

review       False
sentiment    False
dtype: bool

In [11]:
data['sentiment'].value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

In [12]:
data.shape

(50000, 2)

Text Normalization

Tokenization

In [13]:
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud, STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize


In [14]:
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from bs4 import BeautifulSoup 


In [15]:
import nltk

In [16]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [17]:
#Tokenization of text
tokenizers=ToktokTokenizer()
#Setting English stopwords
stopwords=nltk.corpus.stopwords.words('english')

In [18]:
#Removing the noisy text

def noiseremoval_text(text):
  soup=BeautifulSoup(text, "html.parser")
  text =soup.get_text()
  text=re.sub('\[^]]*\]', '', text)
  return text

In [19]:
#Apply function on review column
data['review']=data['review'].apply(noiseremoval_text)

<ipython-input-18-d5f540c5d02a>:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup=BeautifulSoup(text, "html.parser")


In [20]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. The filming tec...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


Stemming

In [21]:
#Stemming the text
def stemmer(text):
  ps=nltk.porter.PorterStemmer()
  text=' '.join([ps.stem(word) for word in text.split(' ')])
  return text

In [22]:
#Apply function on review column
data['review']=data['review'].apply(stemmer)

In [23]:
data.head()

,review,sentiment
0,one of the other review ha mention that after ...,positive
1,a wonder littl production. the film techniqu i...,positive
2,i thought thi wa a wonder way to spend time on...,positive
3,basic there' a famili where a littl boy (jake)...,negative
4,"petter mattei' ""love in the time of money"" is ...",positive


In [24]:
stopword_list = nltk.corpus.stopwords.words('english')
# Removing the stopwords
def removing_stopwords(text, is_lower_case=False, stopwords=stopword_list):
  #Tokenization of text
  tokenizers=ToktokTokenizer()
  #Setting English stopwords
  tokens = tokenizers.tokenize(text)

  tokens = [i.strip() for i in tokens]
  if is_lower_case:
    filtered_tokens = [i for i in tokens if i not in stopwords]
  else:
    filtered_tokens = [i for i in tokens if i.lower() not in stopwords]
  filtered_text=' '.join(filtered_tokens)
  return filtered_text   

In [25]:
#Apply function on review column
data['review']=data['review'].apply(removing_stopwords)

In [26]:
data.head()

,review,sentiment
0,one review ha mention watch 1 oz episod ' hook...,positive
1,wonder littl production. film techniqu veri un...,positive
2,thought thi wa wonder way spend time hot summe...,positive
3,basic ' famili littl boy ( jake ) think ' zomb...,negative
4,"petter mattei ' "" love time money "" visual stu...",positive


Train test Split

In [27]:
#Split the dataset
# train dataset
train_reviews_data=data.review[:30000]

In [28]:
#test dataset
test_reviews_data=data.review[30000:]

Bag of words

In [29]:
#Count vectorizer for bag of words
cv=CountVectorizer(min_df=0, max_df=1, binary=False, ngram_range=(1,3))
#transformed train reviews
cv_train=cv.fit_transform(train_reviews_data)
#transformed test reviews
cv_test=cv.transform(test_reviews_data)

print('BOW_cv_train:', cv_train.shape)
print('BOW_cv_test:', cv_test.shape)


BOW_cv_train: (30000, 4955422)
BOW_cv_test: (20000, 4955422)


TF_IDF

In [30]:
#TFIDF vectorizer
tf =TfidfVectorizer(min_df=0, max_df=1, use_idf=True, ngram_range=(1,3))

tf_train=tf.fit_transform(train_reviews_data)

tf_test=tf.transform(test_reviews_data)

print('Tfidf_train:', tf_train.shape)
print('Tfidf_test:', tf_test.shape)

Tfidf_train: (30000, 4955422)
Tfidf_test: (20000, 4955422)


Label encoding

In [31]:
from sklearn.preprocessing import LabelBinarizer


In [33]:
# Labeling the sentiment data = # Create an instance of the LabelBinarizer class
lb=LabelBinarizer()

# Fit and transform sentiment data

sentiment_data= lb.fit_transform(data['sentiment'])
print(sentiment_data.shape)

(50000, 1)


In [34]:
train_data=data.sentiment[:30000]

In [35]:
test_data=data.sentiment[30000:]

In [36]:
print(train_data.shape)
print(test_data.shape)

(30000,)
(20000,)


In [37]:
#training the model
logistic=LogisticRegression(penalty='l2', max_iter=500, C=1, random_state=42)
# Fitting the model for bag of words
lr_bow=logistic.fit(cv_train, train_data)

In [38]:
print(lr_bow)

LogisticRegression(C=1, max_iter=500, random_state=42)


In [39]:
# Predicting the model for bag of words
bow_predict=logistic.predict(cv_test)
print(bow_predict)

['negative' 'negative' 'negative' ... 'negative' 'negative' 'negative']


In [43]:
#Accuracy score for bag of words
bow_score=accuracy_score(test_data, bow_predict)
print("bow_score:", bow_score)


bow_score: 0.59225


In [40]:
# Fittig the model for tfdif features
lr_tfidf=logistic.fit(tf_train, train_data)
print(lr_tfidf)

LogisticRegression(C=1, max_iter=500, random_state=42)


In [41]:
# Predicting the model for tfidf features
lr_tfidf_predict=logistic.predict(tf_test)
print(lr_tfidf_predict)

['negative' 'negative' 'negative' ... 'negative' 'positive' 'positive']


In [44]:
#Accuracy score for tfidf features
tfidf_score=accuracy_score(test_data, lr_tfidf_predict)
print("tfidf_score:", tfidf_score)


tfidf_score: 0.7428
